# GenWorlds First Steps Tutorial

Welcome to GenWorlds! In this beginner's tutorial, we'll walk through creating a simple GenWorld from scratch. By the end, you'll have a basic understanding of how GenWorlds operates, setting the stage for more advanced use cases.

## Scenario Overview:

Imagine a world where two agents, "John" and "Matthew", work in tandem. John receives a request from a user to generate a random matrix and determine its determinant. John then uses a tool to craft this matrix, which he sends over to Matthew. Matthew, equipped with a different tool, calculates the determinant of the matrix John created. After doing so, Matthew sends back the result to John, who then delivers the original matrix and the determinant value to the user.

While this might sound like overkill for a simple task (since the LLM could probably do this in one go), the purpose is to demonstrate how different components of GenWorlds can interact.

## Step 1: Initial Set Up

Before diving into our world, we need a few basic utilities and configurations.

In [1]:
from datetime import datetime
import threading
from typing import List
from time import sleep
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Step 2: Crafting the World

Our world, named "Compute Matrix Determinant World", will initially be empty. We'll then populate it with agents and objects in subsequent steps.

In [2]:
from genworlds.worlds import World2D

# Define the World
CMD_world = World2D(
    name="Compute Matrix Determinant World",
    description="A world where two agents interact to generate a matrix and compute its determinant.",
    locations=["room1"]
)

## Step 3: Simulation - Bringing the World to Life

Now, let's bring our world to life by creating a simulation and associating it with our world definition.

In [3]:
from genworlds.simulation import Simulation
from genworlds.simulation.sockets.simulation_socket_server import start

# Create the Simulation
CMD_simulation = Simulation(
    name="CMD Simulation", 
    description="A live instance of the CMD World", 
    objects=[], 
    agents=[],
    world=CMD_world
)

# Start the simulation socket in a parallel thread
simulation_socket = threading.Thread(target=start)
simulation_socket.start()

# Wait for the socket to start
sleep(1)

# Launch the simulation
simulation_thread = threading.Thread(target=CMD_simulation.launch)
simulation_thread.start()

INFO:     Started server process [4064]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7456 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 51875) - "WebSocket /ws" [accepted]
INFO:     connection open
[48554608-dde4-4140-9b8f-7c896f5a3a2d Thread] Connected to world socket server ws://127.0.0.1:7456/ws


## Step 4: Introducing the Matrix Generator

To generate matrices, we'll use an object called `MatrixGenerator`. This object will listen to agents' requests to craft matrices and respond accordingly.

In [4]:
from genworlds.objects import BaseObject
from genworlds.events import BaseEvent

# Define the MatrixGenerator Object
class MatrixGenerator(BaseObject):
    def __init__(self):
        self.name = "Matrix Generator"
        self.description = "Object used by agents to generate squared matrices of size N"
        self.id = "matrix_generator"
        self.websocket_url = "ws://127.0.0.1:7456/ws"
        super().__init__(name=self.name, description=self.description, id=self.id, websocket_url=self.websocket_url)

# Instantiate the MatrixGenerator Object
matrix_generator = MatrixGenerator()

# Create two events and one action
# Event that the agent will use to generate a matrix
class AgentGeneratesNxNMatrix(BaseEvent):
    event_type = "agent_generates_n_by_n_matrix"
    description = "An agent generates a squared matrix of size N of integer numbers"
    N: int

# Event that the MatrixGenerator Object will use to give the requested matrix to the agent
class SendGeneratedMatrix(BaseEvent):
    event_type = "send_generated_matrix"
    description = "Sends the requested squared matrix of size N of integer numbers to the agent"
    matrix: List[List[int]]
        
# Function that executes the action of generating a matrix
def agent_generates_n_by_n_matrix(self, event: AgentGeneratesNxNMatrix):
    import numpy as np
    N = event.N
    matrix = np.random.randint(100, size=(N, N))
    print(f"Agent {event.sender_id} has created a matrix of size : {event.N}.")    
    self.send_event(
            SendGeneratedMatrix,
            target_id=event.sender_id,
            matrix = matrix.tolist(),
    )
    
# Attach events and actions to the MatrixGenerator
from types import MethodType
matrix_generator.agent_generates_n_by_n_matrix = MethodType(agent_generates_n_by_n_matrix, matrix_generator)

matrix_generator.register_event_listeners([
    (AgentGeneratesNxNMatrix, matrix_generator.agent_generates_n_by_n_matrix),
])

# Incorporate the Matrix Generator into the Simulation
CMD_simulation.register_object(matrix_generator, world_properties={"location": "room1"})

Registered object id='matrix_generator' entity_type=<EntityTypeEnum.OBJECT: 'OBJECT'> entity_class='MatrixGenerator' name='Matrix Generator' description='Object used by agents to generate squared matrices of size N' held_by=None location=None


INFO:     ('127.0.0.1', 51876) - "WebSocket /ws" [accepted]
INFO:     connection open
[matrix_generator Thread] Connected to world socket server ws://127.0.0.1:7456/ws


## Step 5: Creating a Dummy Agent for Simulation

In this step, we will be creating a simple dummy agent for our simulation using the genworlds library. The agent will have a basic set of goals, a role, and some thought processes attached to it. The main idea behind this step is to understand how to set up an agent with basic functionalities and then integrate it into a given simulation environment.

### Overview:

1. Importing Necessary Modules: Begin by importing the required classes and modules from genworlds.

2. Function for Agent Generation: We will define a reusable function named generate_dummy_agent that will generate an agent based on a given name. The function will configure the agent's role, background, personality, goals, constraints, evaluation principles, and topic of conversation.

3. Agent's Brains: The function will then set up the agent's 'brains' using the NavigationThought and EventFillerThought classes. These 'brains' dictate how the agent thinks and operates in the simulation.

    - Navigation Brain: Responsible for helping the agent navigate through the simulation.

    - Execution Brains: Defines the agent's execution strategies, such as filling in events in this case.

4. Agent Action Brain Map: Defines the default action strategy for the agent.

5. Agent Generation: Using our function, we'll generate an agent named "John" and integrate him into our simulation using CMD_simulation.register_agent.

In [5]:
from genworlds.agents import BaseAgent
from genworlds.agents.base_agent.thoughts.navigation_thought import NavigationThought
from genworlds.agents.base_agent.thoughts.event_filler_thought import EventFillerThought

# Function for generating agents (as we will re-use it later on the tutorial)
def generate_dummy_agent(agent_name):
    """
    Method for generating super simple dummy agents.
    """
    name = agent_name
    role = """Agent that helps the user achieve his goals"""
    background = """"""
    personality = f"""{agent_name} is very diligent."""
    goals = ["Starts waiting and sleeps till the user starts a new question.",
             f"Once {agent_name} receives a user's question, he makes sure to have all the information before sending the answer to the user.",
             f"When {agent_name} has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.",
             "After sending the response, he waits for the next user question."]
    constraints = []
    evaluation_principles = []
    topic_of_conversation = "Help the user."


    return BaseAgent(
        name=name,
        id=name,
        description=f"{role}. {background}",
        goals=goals,
        openai_api_key=openai_api_key,
        wakeup_events={
            "agent_speaks_with_agent_event": {
                "target_id": name
            },
            "user_speaks_with_agent_event": {
                "target_id": name
            },
        },

        navigation_brain = NavigationThought(
            openai_api_key=openai_api_key,
            name=name, 
            role=role,
            background=background,
            personality=personality,
            topic_of_conversation=topic_of_conversation,
            constraints=constraints,
            evaluation_principles=evaluation_principles,
            n_of_thoughts=3,
        ),
        execution_brains={
            "event_filler_brain": EventFillerThought(
                openai_api_key=openai_api_key,
                name=name, 
                role=role,
                background=background,
                topic_of_conversation=topic_of_conversation,
                constraints=constraints,
                evaluation_principles=evaluation_principles,
                n_of_thoughts=1,
            ),
        },
        action_brain_map={
            "default": {"brains":["event_filler_brain"], "next_actions": []},
        },
    )

# Generate a Dummy Agent named John
john = generate_dummy_agent("John")

## Attach John to the Simulation
CMD_simulation.register_agent(john, world_properties={"location": "room1"})

INFO:     ('127.0.0.1', 51877) - "WebSocket /ws" [accepted]
INFO:     connection open
[Agent John Listening Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:     ('127.0.0.1', 51878) - "WebSocket /ws" [accepted]
INFO:     connection open
[Agent John Speaking Thread] Connected to world socket server ws://127.0.0.1:7456/ws


Registered agent id='John' entity_type=<EntityTypeEnum.AGENT: 'AGENT'> entity_class='BaseAgent' name='John' description='Agent that helps the user achieve his goals. ' held_by=None location=None


[John]  The agent John is thinking...
[John] Threads launched


{"event_type": "entity_request_world_state_update_event", "description": "Request the latest world state update for an entity.", "summary": null, "created_at": "2023-09-11T15:37:51.733944", "sender_id": "John", "target_id": "48554608-dde4-4140-9b8f-7c896f5a3a2d"}
{"event_type": "world_sends_schemas_event", "description": "The world sends the possible interactions to all the agents.", "summary": null, "created_at": "2023-09-11T15:37:51.739903", "sender_id": "48554608-dde4-4140-9b8f-7c896f5a3a2d", "target_id": null, "world_name": "Compute Matrix Determinant World", "world_description": "A world where two agents interact to generate a matrix and compute its determinant.", "schemas": {"MatrixGenerator": {"agent_generates_n_by_n_matrix": {"title": "AgentGeneratesNxNMatrix", "type": "object", "properties": {"event_type": {"title": "Event Type", "default": "agent_generates_n_by_n_matrix", "type": "string"}, "description": {"title": "Description", "default": "An agent generates a squared matri


> Finished chain.
Generated: [{'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect the necessary information', "Answer the user's question", 'Wait for another question'], 'next_action_aim': "Wait for user's question", 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'yes'}, {'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect the necessary information', "Answer the user's question", 'Wait for another question'], 'next_action_aim': 'Collect the necessary information', 'next_action': 'MatrixGenerator:agent_generates_n_by_n_matrix', 'violated_constraints': [], 'is_action_valid': 'no'}, {'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect the necessary information', "Answer the user's question", 'Wait for another question'], 'next_action_aim': "Answer the user's question", 'next_action': 'Self:agent_speaks_with_user_event', 'violated_constraints': [], 'is_action_val

[John] {'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect the necessary information', "Answer the user's question", 'Wait for another question'], 'next_action_aim': "Wait for user's question", 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'yes'}
[John] Waiting, entering sleep mode...
[John] Waiting...




> Finished chain.
Evaluated: {'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect the necessary information', "Answer the user's question", 'Wait for another question'], 'next_action_aim': "Wait for user's question", 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'yes'}


[John] Sleeping...
[John] Sleeping...


## Step 6: Simulating User Interaction with the Agent

In this step, we'll demonstrate how to simulate user interaction with the agent in the simulation environment. The objective is to have a pseudo-user send a request to our dummy agent "John", asking him to perform specific tasks. This helps in understanding the dynamics of agent-user communication and to observe how the agent reacts and processes user's requests.

In [7]:
from uuid import uuid4
from genworlds.simulation.sockets.simulation_socket_client import SimulationSocketClient
from genworlds.events.basic_events import UserSpeaksWithAgentEvent

class FakeUser:
    def __init__(
        self,
        name: str,
        description: str,
        id: str = None,
        websocket_url: str = "ws://127.0.0.1:7456/ws",
    ):
        self.id = id if id else str(uuid4())
        self.name = name
        self.description = description
        self.socket_client = SimulationSocketClient(process_event=lambda x: print("\n"), url=websocket_url)
        threading.Thread(
                    target=self.socket_client.websocket.run_forever,
                    name=f"{self.id} Thread",
                    daemon=True,
                ).start()
        
# Create the Fake User
jm = FakeUser("JM", "Me", "JM")

# Format the message that will be sent to the simulation socket
test_msg = "Hey John, generate a 4x4 matrix and send it to me please!"
message_to_send = UserSpeaksWithAgentEvent(
    sender_id=jm.id, 
    created_at=datetime.now(), 
    message=test_msg, 
    target_id="John"
).json()

sleep(1)

# Send the message to John
jm.socket_client.send_message(message_to_send)

INFO:     ('127.0.0.1', 51881) - "WebSocket /ws" [accepted]
INFO:     connection open
[JM Thread] Connected to world socket server ws://127.0.0.1:7456/ws


{"event_type": "user_speaks_with_agent_event", "description": "The user speaks with an agent.", "summary": null, "created_at": "2023-09-11T15:39:09.860192", "sender_id": "JM", "target_id": "John", "message": "Hey John, generate a 4x4 matrix and send it to me please!"}






[John] Waking up


{'functions': [{'name': 'generate_options', 'description': 'Generates 3 options for the agent to choose from.', 'parameters': {'type': 'object', 'properties': {'options': {'type': 'array', 'minItems': 3, 'maxItems': 3, 'items': {'type': 'object', 'properties': {'inventory': {'type': 'array', 'description': 'list the ids of the items in your inventory', 'items': {'type': 'string'}}, 'step_completed': {'type': 'string', 'description': 'If you completed a step of your plan, put it here, as well as an explanation of how you did it'}, 'plan': {'type': 'array', 'description': 'A numbered list of an updated plan', 'items': {'type': 'string'}}, 'next_action_aim': {'type': 'string', 'description': 'What is the aim of the next action you want to take?'}, 'next_action': {'type': 'string', 'enum': ['Self:wait', 'Self:agent_speaks_with_user_event', 'MatrixGenerator:agent_generates_n_by_n_matrix', 'World:agent_gives_object_to_agent_event', 'World:agent_moves_to_new_location'], 'description': 'What i

[John] {'inventory': [], 'step_completed': 'None', 'plan': ['Generate a 4x4 matrix', 'Send the matrix to the user', "Wait for the next user's question"], 'next_action_aim': 'Generate a 4x4 matrix', 'next_action': 'MatrixGenerator:agent_generates_n_by_n_matrix', 'violated_constraints': [], 'is_action_valid': 'Yes'}


...

[John] Event at: 2023-09-11T15:41:08.131715
What happened: John, the sender, spoke with user JM and sent a message containing a series of numbers.



{'functions': [{'name': 'generate_options', 'description': 'Generates 3 options for the agent to choose from.', 'parameters': {'type': 'object', 'properties': {'options': {'type': 'array', 'minItems': 3, 'maxItems': 3, 'items': {'type': 'object', 'properties': {'inventory': {'type': 'array', 'description': 'list the ids of the items in your inventory', 'items': {'type': 'string'}}, 'step_completed': {'type': 'string', 'description': 'If you completed a step of your plan, put it here, as well as an explanation of how you did it'}, 'plan': {'type': 'array', 'description': 'A numbered list of an updated plan', 'items': {'type': 'string'}}, 'next_action_aim': {'type': 'string', 'description': 'What is the aim of the next action you want to take?'}, 'next_action': {'type': 'string', 'enum': ['Self:wait', 'Self:agent_speaks_with_user_event', 'MatrixGenerator:agent_generates_n_by_n_matrix', 'World:agent_gives_object_to_agent_event', 'World:agent_moves_to_new_location'], 'description': 'What i

[John] {'inventory': [], 'step_completed': 'Send the matrix to the user', 'plan': ["Wait for the next user's question"], 'next_action_aim': 'Wait for the user to start a new question', 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'Yes'}
[John] Waiting, entering sleep mode...
[John] Waiting...




> Finished chain.
Evaluated: {'inventory': [], 'step_completed': 'Send the matrix to the user', 'plan': ["Wait for the next user's question"], 'next_action_aim': 'Wait for the user to start a new question', 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'Yes'}


[John] Sleeping...
[John] Sleeping...


## Step 7: Integrating a Determinant Calculator into the Simulation

In this step, we will introduce an object into our simulation environment that agents can utilize to compute the determinant of matrices. This serves as a representation of how tools and utilities can be made available to agents, enhancing their capabilities.

In [8]:
# Define the DetCalculator Object
class DetCalculator(BaseObject):
    def __init__(self):
        self.name = "Determinant Calculator"
        self.description = "Object used by agents to calculate determinants of a matrix."
        self.id = "det_calculator"
        self.websocket_url = "ws://127.0.0.1:7456/ws"
        super().__init__(name=self.name, description=self.description, id=self.id, websocket_url=self.websocket_url)

# Instantiate it
det_calculator = DetCalculator()

# Create two events and one action
# Event that the agent will use to compute the determinant of a given matrix
class AgentComputesDeterminant(BaseEvent):
    event_type = "agent_computes_determinant"
    description = "An agent computes the determinant of a matrix"
    matrix: List[List[int]]

# Event that the DetCalculator Object will use to give the requested determinant to the agent
class SendMatrixDeterminant(BaseEvent):
    event_type = "send_matrix_determinant"
    description = "Sends the requested determinant of the matrix to the agent"
    determinant: int

# Function that executes the action of computing a determinant
def agent_computes_determinant(self, event: AgentComputesDeterminant):
    import numpy as np
    determinant = np.linalg.det(np.array(event.matrix))
    print(f"Agent {event.sender_id} has computed the determinant of : {event.matrix}.")    
    self.send_event(
            SendMatrixDeterminant,
            target_id=event.sender_id,
            determinant = determinant,
    )

# Attach events and actions to the DetCalculator
from types import MethodType
det_calculator.agent_computes_determinant = MethodType(agent_computes_determinant, det_calculator)

det_calculator.register_event_listeners([
    (AgentComputesDeterminant, det_calculator.agent_computes_determinant),
])

# Incorporate the Matrix Generator into the Simulation
CMD_simulation.register_object(det_calculator, world_properties={"location": "room1"})

Registered object id='det_calculator' entity_type=<EntityTypeEnum.OBJECT: 'OBJECT'> entity_class='DetCalculator' name='Determinant Calculator' description='Object used by agents to calculate determinants of a matrix.' held_by=None location=None


INFO:     ('127.0.0.1', 51906) - "WebSocket /ws" [accepted]
INFO:     connection open
[det_calculator Thread] Connected to world socket server ws://127.0.0.1:7456/ws
[John] Sleeping...


## Step 8: Introducing Another Agent and Requesting World State Updates
In this step, we will be adding a second agent, "Matthew", to our simulation. Both John and Matthew will then request updates on the world state, allowing them to become aware of each other and other changes in the simulation environment.

In [9]:
matthew = generate_dummy_agent("Matthew")
CMD_simulation.register_agent(matthew, world_properties={"location": "room1"})

# Requesting world updates
john.agent_request_world_state_update_action()
matthew.agent_request_world_state_update_action()

INFO:     ('127.0.0.1', 51907) - "WebSocket /ws" [accepted]
INFO:     connection open
[Agent Matthew Listening Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:     ('127.0.0.1', 51908) - "WebSocket /ws" [accepted]
INFO:     connection open
[Agent Matthew Speaking Thread] Connected to world socket server ws://127.0.0.1:7456/ws
[Matthew]  The agent Matthew is thinking...


Registered agent id='Matthew' entity_type=<EntityTypeEnum.AGENT: 'AGENT'> entity_class='BaseAgent' name='Matthew' description='Agent that helps the user achieve his goals. ' held_by=None location=None


[Matthew] Threads launched


{"event_type": "entity_request_world_state_update_event", "description": "Request the latest world state update for an entity.", "summary": null, "created_at": "2023-09-11T15:42:05.580725", "sender_id": "Matthew", "target_id": "48554608-dde4-4140-9b8f-7c896f5a3a2d"}




{"event_type": "entity_request_world_state_update_event", "description": "Request the latest world state update for an entity.", "summary": null, "created_at": "2023-09-11T15:42:05.582978", "sender_id": "John", "target_id": "48554608-dde4-4140-9b8f-7c896f5a3a2d"}
{"event_type": "entity_request_world_state_update_event", "description": "Request the latest world state update for an entity.", "summary": null, "created_at": "2023-09-11T15:42:05.583978", "sender_id": "Matthew", "target_id": "48554608-dde4-4140-9b8f-7c896f5a3a2d"}




{"event_type": "world_sends_schemas_event", "description": "The world sends the possible interactions to all the agents.", "summary": null, "created_at": "2023-09-11T15:42:05.586978", "sender_id

[John] Sleeping...
[John] Sleeping...


...

[John] Sleeping...
[John] Sleeping...
[John] Sleeping...
[John] Sleeping...
[John] Sleeping...



> Finished chain.
Generated: [{'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect required information', 'Respond to user', 'Wait for next question'], 'next_action_aim': "Wait for user's question", 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'Yes'}, {'inventory': [], 'step_completed': '', 'plan': ['Generate matrix', 'Calculate determinant', 'Respond to user'], 'next_action_aim': 'Generate matrix', 'next_action': 'MatrixGenerator:agent_generates_n_by_n_matrix', 'violated_constraints': [], 'is_action_valid': 'Yes'}, {'inventory': [], 'step_completed': '', 'plan': ['Move to new location', "Wait for user's question", 'Collect required information', 'Respond to user'], 'next_action_aim': 'Move to new location', 'next_action': 'World:agent_moves_to_new_location', 'violated_constraints': [], 'is_action_valid': 'Yes'}]


> Entering new LLMChain chain...
Prompt after formatting:
System: 
# Basic rules
You are Matthew, Agent that 

[John] Sleeping...
[Matthew] {'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect required information', 'Respond to user', 'Wait for next question'], 'next_action_aim': "Wait for user's question", 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'Yes'}
[Matthew] Waiting, entering sleep mode...
[Matthew] Waiting...

[John] Sleeping...



> Finished chain.
Evaluated: {'inventory': [], 'step_completed': '', 'plan': ["Wait for user's question", 'Collect required information', 'Respond to user', 'Wait for next question'], 'next_action_aim': "Wait for user's question", 'next_action': 'Self:wait', 'violated_constraints': [], 'is_action_valid': 'Yes'}


[Matthew] Sleeping...
[John] Sleeping...
[Matthew] Sleeping...
[John] Sleeping...
[Matthew] Sleeping...
[John] Sleeping...
[Matthew] Sleeping...
[John] Sleeping...
[Matthew] Sleeping...
[John] Sleeping...
[Matthew] Sleeping...
[John] Sleeping...
[Matthew] Sleeping...
[John] Sleeping...
[Matthew] Sleeping...


## Step 9: Final Collaborative Test between Agents

In the final step of this tutorial, we're putting everything together to demonstrate how agents can collaborate in the simulation world. We'll instruct John to generate a 3x3 matrix, have its determinant computed by Matthew, and then relay the matrix and its determinant back to us, the user.

In [10]:
test_msg = """Hey John, 
generate a matrix 3x3, 
send it to Matthew to compute its determinant, 
and when he replies back to you, tell me the matrix and its determinant."""

message_to_send = UserSpeaksWithAgentEvent(
    sender_id=jm.id, 
    created_at=datetime.now(), 
    message=test_msg, 
    target_id="John"
).json()

jm.socket_client.send_message(message_to_send)

{"event_type": "user_speaks_with_agent_event", "description": "The user speaks with an agent.", "summary": null, "created_at": "2023-09-11T15:43:24.300474", "sender_id": "JM", "target_id": "John", "message": "Hey John, \ngenerate a matrix 3x3, \nsend it to Matthew to compute its determinant, \nand when he replies back to you, tell me the matrix and its determinant."}






[Matthew] Sleeping...
[John] Waking up


{'functions': [{'name': 'generate_options', 'description': 'Generates 3 options for the agent to choose from.', 'parameters': {'type': 'object', 'properties': {'options': {'type': 'array', 'minItems': 3, 'maxItems': 3, 'items': {'type': 'object', 'properties': {'inventory': {'type': 'array', 'description': 'list the ids of the items in your inventory', 'items': {'type': 'string'}}, 'step_completed': {'type': 'string', 'description': 'If you completed a step of your plan, put it here, as well as an explanation of how you did it'}, 'plan': {'type': 'array', 'description': 'A numbered list of an updated plan', 'items': {'type': 'string'}}, 'next_action_aim': {'type': 'string', 'description': 'What is the aim of the next action you want to take?'}, 'next_action': {'type': 'string', 'enum': ['Self:wait', 'Self:agent_speaks_with_user_event', 'MatrixGenerator:agent_generates_n_by_n_matrix', 'DetCalculator:agent_computes_determinant', 'World:agent_gives_object_to_agent_event', 'World:agent_mov

[Matthew] Sleeping...
[Matthew] Sleeping...
[Matthew] Sleeping...


Prompt after formatting:
System: 
# Basic rules

You are John, Agent that helps the user achieve his goals. You need to come up with the next step to get you closer to your goals. It must be consistent with all of the following information:

## Your background


## Your personality
John is very diligent.

## Topic of conversation
Help the user.


## Your Goals
1. Starts waiting and sleeps till the user starts a new question.
2. Once John receives a user's question, he makes sure to have all the information before sending the answer to the user.
3. When John has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.
4. After sending the response, he waits for the next user question.

## World State
You are an agent in a 2D world.
THe id of the world is "48554608-dde4-4140-9b8f-7c896f5a3a2d".
Your id is "John".
Your location is "None".
Available locations are "['room1']".

        
System: The current time and date is Mon Sep 11 15:

[Matthew] Sleeping...


## Conclusions

Congratulations on reaching the end of this tutorial on simulating agent collaboration in the GenWorlds environment! Let's reflect on what we've accomplished and what we've learned:

### Key Takeaways:
1. **Basics of the GenWorlds Framework:**
    - We began with a solid foundation, introducing the core components and functionalities of the GenWorlds simulation framework. This framework is powerful and flexible, enabling the creation and management of complex agent-driven worlds.
2. **Creation of Dummy Agents:**
    - We went through the process of creating basic, dummy agents and setting up their attributes. This gave us a hands-on understanding of agent attributes like name, role, background, and their thought processes like navigation_brain.
3. **User-Agent Interaction:**
    - We designed a Fake User to simulate real-world interactions between users and agents in the environment. This user-agent interaction plays a crucial role in instructing agents and receiving feedback from them.
4. **Defining Custom Events & Objects:**
    - Our tutorial introduced the concept of custom events, allowing us to define specific actions like matrix determinant computation. We also learned about objects in the simulation, such as our "Determinant Calculator."
5. **Inter-Agent Collaboration:**
    - One of the highlights was demonstrating how agents can collaborate. We instructed John to create a matrix, which was then processed by Matthew, showcasing the potential for multi-agent workflows.

### Further Exploration:
While we've covered a lot, the GenWorlds environment offers even more to explore:
- **Complex Agent Behaviors:** Our tutorial focused on basic, dummy agents. Delve deeper into the framework to create agents with intricate behaviors, decision-making processes, and reactions to diverse events.
- **Expand the World:** Introduce more objects, locations, and complexities to your simulation. Imagine scenarios like agents navigating through a maze or collaborating to solve puzzles.
- **Real-time Analytics:** With multiple agents and complex events, the simulation can produce vast amounts of data. Dive into analyzing this data to gain insights into agent behaviors, interactions, and more.

### Wrapping Up:
Agent-based simulations like GenWorlds are powerful tools for modeling complex systems and interactions. They offer insights into multi-agent behaviors, decision-making processes, and collaborative efforts. By mastering these simulations, you'll be equipped to tackle intricate problems, design intelligent systems, and model real-world scenarios with confidence.

Thank you for journeying through this tutorial with us. We hope it has sparked your interest in exploring the vast possibilities of agent-based simulations further!